In [103]:
import pandas as pd
import numpy as np
import math
from scipy import signal
import matplotlib.pyplot as plt

In [104]:
PATH_MAIN = 'C:/EEG_Embedded_Systems/WhiteNoise/DataTrainWhiteNoise/0.5/'
PATH_7Hz = PATH_MAIN + '7Hz'
PATH_9Hz = PATH_MAIN + '9Hz'
PATH_11Hz = PATH_MAIN + '11Hz'
PATH_13Hz = PATH_MAIN + '13Hz'
PATH_Baseline = PATH_MAIN + 'Baseline'

DEST_PATH = 'C:/EEG_Embedded_Systems/WhiteNoise/Features/train/'

ELECTRODOS = ['EEG.O1', 'EEG.O2']

In [105]:
def leerEtiquetas(carpeta):
  if carpeta == '7Hz':
    etiqueta = 1
  elif carpeta == '9Hz':
    etiqueta = 2
  elif carpeta == '11Hz':
    etiqueta = 3
  elif carpeta == '13Hz':
    etiqueta = 4
  elif carpeta == 'Baseline':
    etiqueta = 5
  return etiqueta

In [106]:
def leerArchivo(path, num_archivo):
    datos = pd.read_csv(path + '/muestra' + num_archivo + '.csv')
    return datos

In [107]:
def valorAbsolutoMedio(datos, columna):
    lista = []
    #datos = leerArchivo(path, num_archivo)
    return abs((datos)).to_frame().mean()[columna]
    

In [108]:
def valorAbsolutoPromedioWeightsI(datos):
  lista = []
  #datos = leerArchivo(path, num_archivo)
  for fila in range(len(datos)):
    fila += 1
    if fila <= 0.75*len(datos) and fila >= 0.25*len(datos):
      w = 1
    else:
      w = 0.5
    lista.append(abs(datos.iloc[fila-1])*w)
  return np.array(lista).mean()

In [109]:
def valorAbsolutoPromedioWeightsII(datos):
  lista = []
  #datos = leerArchivo(path, num_archivo)
  for fila in range(len(datos)):
    fila += 1
    if fila <= 0.75*len(datos) and fila >= 0.25*len(datos):
      w = 1
    elif fila < 0.25*len(datos) :
      w = 4*fila/len(datos)
    else:
      w = 4*(len(datos)-fila)/len(datos)
    lista.append(abs(datos.iloc[fila-1])*w)
  return np.array(lista).mean()

In [110]:
def detectorRegistro(datos, columna):
  #datos = leerArchivo(path, num_archivo)
  return math.exp(np.log(abs(datos).to_frame()).mean()[columna])

In [111]:
def ValorAbsolutoMediana(datos, columna):
  #datos = leerArchivo(path, num_archivo)
  return abs((datos)).to_frame().median()[columna]
  

In [112]:
def varianza(datos):
  #datos = leerArchivo(path, num_archivo)
  return abs((datos)).var(ddof=0)

In [113]:
def diferencia_absoluta(v):
  sum_diff = 0
  for i,x in enumerate(v):
    if i+1 < len(v):
      sum_diff += abs(v[i+1] - x)
      r = sum_diff/len(v)
  return r
def valorMediaDiferenciaAbsoluta(datos):
  #datos = leerArchivo(path, num_archivo)
  datos = datos.to_list()
  return diferencia_absoluta(datos)

In [114]:
def zeroCrossing(datos):
  #datos = leerArchivo(path, num_archivo)[columna].to_list()
  datos = datos.to_list()
  zerocrossing = 0
  for i in range(1, len(datos)):
    if ((datos[i-1]) > 0 and datos[i] < 0):
        zerocrossing += 1
    if ((datos[i-1]) < 0 and datos[i] > 0):
        zerocrossing += 1
  return zerocrossing

In [115]:
def frecuenciaMedia(datos):
  #datos = leerArchivo(path, num_archivo)[columna].to_list()
  datos = datos.to_list()
  frecuencia, spectrum = signal.periodogram(datos, 128)
  frecuenciaMedia = ((frecuencia * spectrum).sum())/(spectrum.sum())
  return frecuenciaMedia

In [116]:
def frecuenciaPico(datos):
  #datos = leerArchivo(path, num_archivo)[columna].to_list()
  datos = datos.to_list()
  f, sp = signal.periodogram(datos, 128)
  frecuencia = f[30:60]
  spectrum = sp[30:60]
  frecuencia_pico_indice = np.argmax(spectrum)
  return frecuencia[frecuencia_pico_indice]

In [117]:
def VarianzaFrecuenciaCentral(datos):
    #datos = leerArchivo(path, num_archivo)[columna].to_list()
    datos = datos.to_list()
    f, sp = signal.periodogram(datos, 128)
    frecuencia_promedio = np.mean(f)
    return np.sum(sp * pow(f- frecuencia_promedio,2))/frecuencia_promedio

In [118]:
def PSDmax(datos):
  #datos = leerArchivo(path, num_archivo)
  #datos = datos[columna].to_list()
  datos = datos.to_list()
  f, sp = signal.periodogram(datos, 128)
  frecuencia = f[30:60]
  spectrum = sp[30:60]
  psd_max = np.max(spectrum)
  return psd_max

In [119]:
df = pd.DataFrame()
etiquetas = []
for electrodo in ELECTRODOS:
    etiquetas = []
    abs_mean = []
    abs_mean_I = []
    abs_mean_II = []
    log_det = []
    abs_median = []
    variance = []
    diff_abs_mean = []
    frec_mean = []
    frec_PSD_max = []
    variance_frec_centr = []
    PSD_max = []
    for folder in os.listdir(PATH_MAIN):
        print(folder)
        src_folder = os.path.join(PATH_MAIN, folder)
        files = [e for e in os.listdir(src_folder) if e.endswith('.csv')]
        for index, file_ in enumerate(files):
            src_file = os.path.join(src_folder, file_)
            df_file = pd.read_csv(src_file)
            abs_mean.append(valorAbsolutoMedio(df_file[electrodo], electrodo))
            abs_mean_I.append(valorAbsolutoPromedioWeightsI(df_file[electrodo]))
            abs_mean_II.append(valorAbsolutoPromedioWeightsII(df_file[electrodo])) 
            log_det.append(detectorRegistro(df_file[electrodo], electrodo))
            abs_median.append(ValorAbsolutoMediana(df_file[electrodo], electrodo))
            variance.append(varianza(df_file[electrodo]))
            diff_abs_mean.append(valorMediaDiferenciaAbsoluta(df_file[electrodo]))
            frec_mean.append(frecuenciaMedia(df_file[electrodo]))
            frec_PSD_max.append(frecuenciaPico(df_file[electrodo]))
            variance_frec_centr.append(VarianzaFrecuenciaCentral(df_file[electrodo]))
            etiquetas.append(leerEtiquetas(folder))
            PSD_max.append(PSDmax(df_file[electrodo]))
    df['valorAbsMedio_' + electrodo] = abs_mean
    df['valorAbsPromWI_' + electrodo] = abs_mean_I
    df['valorAbsPromWII_' + electrodo] = abs_mean_II
    df['LogDet_' + electrodo] = log_det
    df['valorAbsMediana_' + electrodo] = abs_median
    df['Varianza_' + electrodo] = variance
    df['valorMediaDiferenciaAbs_' + electrodo] = diff_abs_mean
    df['frecMedia_' + electrodo] = frec_mean
    df['Frecuencia_en_PSDMAX_' + electrodo] = frec_PSD_max
    df['VarianzaFrecCentral_' + electrodo] = variance_frec_centr
    df['PSD_max_' + electrodo] = PSD_max
#df['Etiquetas'] = etiquetas


11Hz
13Hz
7Hz
9Hz
Baseline
11Hz
13Hz
7Hz
9Hz
Baseline


In [120]:
df

,valorAbsMedio_EEG.O1,valorAbsPromWI_EEG.O1,valorAbsPromWII_EEG.O1,LogDet_EEG.O1,valorAbsMediana_EEG.O1,Varianza_EEG.O1,valorMediaDiferenciaAbs_EEG.O1,frecMedia_EEG.O1,Frecuencia_en_PSDMAX_EEG.O1,VarianzaFrecCentral_EEG.O1,...,valorAbsPromWI_EEG.O2,valorAbsPromWII_EEG.O2,LogDet_EEG.O2,valorAbsMediana_EEG.O2,Varianza_EEG.O2,valorMediaDiferenciaAbs_EEG.O2,frecMedia_EEG.O2,Frecuencia_en_PSDMAX_EEG.O2,VarianzaFrecCentral_EEG.O2,PSD_max_EEG.O2
0,4167.793503,3125.867655,3122.650448,4167.769880,4167.548337,197.111341,4.086688,3.499429,10.595078,18852.577175,...,3130.017782,3125.574587,4175.062592,4172.684081,504.915800,5.241585,2.345228,16.035794,50424.428709,14.518800
1,4167.118150,3123.218496,3125.435944,4167.110516,4167.219163,63.624857,7.440480,23.254776,15.394209,1705.769280,...,3121.195349,3123.333276,4164.554571,4163.902199,94.024499,7.320164,15.985571,9.692650,4755.880087,6.829263
2,4166.469116,3128.579544,3125.808142,4166.453105,4167.798786,133.481084,2.963003,3.245516,10.784922,12844.257723,...,3128.385663,3125.664399,4164.757477,4162.196252,322.500101,3.458289,2.083221,11.068736,32701.399954,6.769330
3,4164.831522,3123.238525,3123.469726,4164.821592,4167.072090,82.679591,5.161281,11.516348,16.932735,5078.396687,...,3123.199174,3123.829927,4164.559417,4164.720699,136.641081,6.216914,10.067647,10.331839,8891.419701,10.779343
4,4166.189718,3125.868283,3125.714814,4166.152413,4166.568429,311.158775,5.561154,3.701353,8.609865,29549.947619,...,3129.344890,3129.626647,4172.293656,4171.925017,198.798055,5.883314,6.293004,9.183857,16744.910957,3.135260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,4170.617815,3125.264098,3127.412571,4170.610913,4170.405759,57.576178,2.682492,6.445014,16.534521,4656.702814,...,3121.457752,3123.522617,4165.494144,4167.907994,134.778523,4.029532,5.116568,8.837416,11768.528341,3.274585
2639,4156.277642,3121.512568,3120.082276,4156.180397,4154.087436,806.568220,3.474246,1.480326,8.876957,82836.581338,...,3133.410698,3132.262161,4172.014495,4171.304090,792.621445,3.964640,1.693596,10.881432,80739.138771,5.410581
2640,4167.702657,3126.494428,3125.750212,4167.568262,4170.480240,1118.711430,3.816910,1.175480,9.102222,118047.332002,...,3131.126137,3130.898443,4172.644762,4170.732314,1491.963425,4.483120,1.229689,9.102222,156955.550693,7.150045
2641,4160.715675,3122.389907,3120.162690,4160.692795,4161.163550,190.319687,3.491192,3.099731,11.352550,18495.905185,...,3124.189706,3121.543719,4163.708263,4166.337132,165.447737,3.868773,4.506763,11.352550,14868.800347,8.703453


In [121]:
def HistogramaAmplitud(datos):
  interv1_lista = []
  interv2_lista = []
  interv3_lista = []
  interv4_lista = []
  interv5_lista = []
  interv6_lista = []
  interv7_lista = []
  interv8_lista = []
  interv9_lista = []
  interv10_lista = []
  #datos = leerArchivo(path, num_archivo)[columna]
  frecuencia, spectrum = signal.periodogram(datos, 128)
  datos = frecuencia
  spectrum = spectrum.tolist()
  total = sum(spectrum)
  for dato in datos:
    if interv1[0] <= dato < interv1[1]:
      interv1_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv2[0] <= dato < interv2[1]:
      interv2_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv3[0] <= dato < interv3[1]:
      interv3_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv4[0] <= dato < interv4[1]:
      interv4_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv5[0] <= dato < interv5[1]:
      interv5_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv6[0] <= dato < interv6[1]:
      interv6_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv7[0] <= dato < interv7[1]:
      interv7_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv8[0] <= dato < interv8[1]:
      interv8_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv9[0] <= dato < interv9[1]:
      interv9_lista.append(spectrum[int(np.where(datos == dato)[0])])
    elif interv10[0] <= dato < interv10[1]:
      interv10_lista.append(spectrum[int(np.where(datos == dato)[0])])
  primer_intervalo.append(sum(interv1_lista)/total)
  segundo_intervalo.append(sum(interv2_lista)/total)
  tercer_intervalo.append(sum(interv3_lista)/total)
  cuarto_intervalo.append(sum(interv4_lista)/total)
  quinto_intervalo.append(sum(interv5_lista)/total)
  sexto_intervalo.append(sum(interv6_lista)/total)
  septimo_intervalo.append(sum(interv7_lista)/total)
  octavo_intervalo.append(sum(interv8_lista)/total)
  noveno_intervalo.append(sum(interv9_lista)/total)
  decimo_intervalo.append(sum(interv10_lista)/total)

In [122]:
for electrodo in ELECTRODOS:
        #DEFINIR LOS INTERVALOS DE FRECUENCIA
    interv1 = [0,6.5]
    interv2 = [interv1[1], interv1[1]+6.5]
    interv3 = [interv2[1], interv2[1]+6.5]
    interv4 = [19.5, 26]
    interv5 = [interv4[1], interv4[1]+6.5]
    interv6 = [interv5[1], interv5[1]+6.5]
    interv7 = [interv6[1], interv6[1]+6.5]
    interv8 = [interv7[1], interv7[1]+6.5]
    interv9 = [interv8[1], interv8[1]+6.5]
    interv10 = [interv9[1], interv9[1]+6.5]
    
    primer_intervalo = []
    segundo_intervalo = []
    tercer_intervalo = []
    cuarto_intervalo = []
    quinto_intervalo = []
    sexto_intervalo = []
    septimo_intervalo = []
    octavo_intervalo = []
    noveno_intervalo = []
    decimo_intervalo = []
    for folder in os.listdir(PATH_MAIN):
        print(folder)
        src_folder = os.path.join(PATH_MAIN, folder)
        files = [e for e in os.listdir(src_folder) if e.endswith('.csv')]
        for index, file_ in enumerate(files):
            src_file = os.path.join(src_folder, file_)
            df_file = pd.read_csv(src_file)
            HistogramaAmplitud(df_file[electrodo])
    df['FHist1_' + electrodo] = primer_intervalo
    df['FHist2_'+ electrodo] = segundo_intervalo
    df['FHist3_'+ electrodo] = tercer_intervalo
    df['FHist4_'+ electrodo] = cuarto_intervalo
    df['FHist5_'+ electrodo] = quinto_intervalo
    df['FHist6_'+ electrodo] = sexto_intervalo
    df['FHist7_'+ electrodo] = septimo_intervalo
    df['FHist8_'+ electrodo] = octavo_intervalo
    df['FHist9_'+ electrodo] = noveno_intervalo
    df['FHist10_'+ electrodo] = decimo_intervalo

11Hz
13Hz
7Hz
9Hz
Baseline
11Hz
13Hz
7Hz
9Hz
Baseline


In [123]:
df

,valorAbsMedio_EEG.O1,valorAbsPromWI_EEG.O1,valorAbsPromWII_EEG.O1,LogDet_EEG.O1,valorAbsMediana_EEG.O1,Varianza_EEG.O1,valorMediaDiferenciaAbs_EEG.O1,frecMedia_EEG.O1,Frecuencia_en_PSDMAX_EEG.O1,VarianzaFrecCentral_EEG.O1,...,FHist1_EEG.O2,FHist2_EEG.O2,FHist3_EEG.O2,FHist4_EEG.O2,FHist5_EEG.O2,FHist6_EEG.O2,FHist7_EEG.O2,FHist8_EEG.O2,FHist9_EEG.O2,FHist10_EEG.O2
0,4167.793503,3125.867655,3122.650448,4167.769880,4167.548337,197.111341,4.086688,3.499429,10.595078,18852.577175,...,0.910947,0.021693,0.034661,0.011904,0.009945,0.007279,0.003119,0.000202,0.000144,0.000106
1,4167.118150,3123.218496,3125.435944,4167.110516,4167.219163,63.624857,7.440480,23.254776,15.394209,1705.769280,...,0.374021,0.138948,0.094536,0.106834,0.094257,0.127814,0.055970,0.006208,0.000866,0.000547
2,4166.469116,3128.579544,3125.808142,4166.453105,4167.798786,133.481084,2.963003,3.245516,10.784922,12844.257723,...,0.932572,0.032796,0.009126,0.007634,0.008219,0.005588,0.003597,0.000245,0.000121,0.000102
3,4164.831522,3123.238525,3123.469726,4164.821592,4167.072090,82.679591,5.161281,11.516348,16.932735,5078.396687,...,0.557254,0.131474,0.130612,0.064940,0.051764,0.049885,0.012676,0.000678,0.000386,0.000332
4,4166.189718,3125.868283,3125.714814,4166.152413,4166.568429,311.158775,5.561154,3.701353,8.609865,29549.947619,...,0.784492,0.054647,0.026807,0.031438,0.040425,0.048848,0.011962,0.000996,0.000291,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,4170.617815,3125.264098,3127.412571,4170.610913,4170.405759,57.576178,2.682492,6.445014,16.534521,4656.702814,...,0.822615,0.062621,0.035247,0.022202,0.020442,0.023511,0.011812,0.001004,0.000361,0.000184
2639,4156.277642,3121.512568,3120.082276,4156.180397,4154.087436,806.568220,3.474246,1.480326,8.876957,82836.581338,...,0.962477,0.015443,0.007696,0.004920,0.003813,0.003625,0.001287,0.000370,0.000191,0.000177
2640,4167.702657,3126.494428,3125.750212,4167.568262,4170.480240,1118.711430,3.816910,1.175480,9.102222,118047.332002,...,0.978595,0.009065,0.003079,0.002932,0.002048,0.002423,0.001622,0.000142,0.000067,0.000026
2641,4160.715675,3122.389907,3120.162690,4160.692795,4161.163550,190.319687,3.491192,3.099731,11.352550,18495.905185,...,0.826908,0.083400,0.026748,0.012136,0.022648,0.021843,0.005343,0.000408,0.000289,0.000277


In [124]:
df['Etiqueta'] = etiquetas

In [127]:
df.to_csv(DEST_PATH + 'Features_0.5_RO.csv', index = False)

In [ ]:
minimos = []
maximos = []
for i in range(len(colores)):
  for num_archivo in range(colores[i]+1):
    datos = leerArchivo(paths[i], str(num_archivo))['EEG.O1'].to_list()
    frecuencia, spectrum = signal.periodogram(datos, 128)